In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [29]:
import json
import yaml

In [30]:
with open('/content/gdrive/MyDrive/A1b-ChatGPT/data.yaml', 'r') as file:
  data = yaml.safe_load(file)

data

{'categories': ['depression'],
 'conversations': [['What Is Depression?',
   'Depression is a common and serious medical illness that negatively affects how you feel, the way you think and how you act.',
   'Fortunately,it is also treatable.',
   'Depression causes feelings of sadness and/or a loss of interest in activities you once enjoyed. It can lead to a variety of emotional and physical problems and can decrease your ability to function at work and at home.'],
  ['I feel i have let my parents down',
   'No matter what,your parents will always be proud of you and will love you.',
   'You will feel much better if you share your feelings with them.'],
  ['What are the types of depression?',
   'There are many different forms of depression but most common types are.',
   'Clinical depression,persistent depressive disorder,bipolar disorder,postnatal disorder.'],
  ['I am good for nothing.',
   "Don't ever think you cannot achieve what you want.",
   'You are capable of reaching your go

In [31]:
convos = data['conversations']

convos[0]

['What Is Depression?',
 'Depression is a common and serious medical illness that negatively affects how you feel, the way you think and how you act.',
 'Fortunately,it is also treatable.',
 'Depression causes feelings of sadness and/or a loss of interest in activities you once enjoyed. It can lead to a variety of emotional and physical problems and can decrease your ability to function at work and at home.']

In [32]:
output = []

In [33]:
for convo in convos:
  completion = ''
  for i, dialog in enumerate(convo):
    if i == 0:
      prompt = dialog
      # p_encode = prompt.encode("ascii", "ignore")
      # prompt = p_encode.decode()
      prompt = prompt.replace("\xa0", " ")
      # print('prompt:',prompt)
    else:
      completion += " " + dialog
      # c_encode = completion.encode("ascii", "ignore")
      # completion = c_encode.decode()
      completion = completion.replace("\xa0", " ")
  completion = completion.strip()
  line = {'prompt': prompt, 'completion': completion}
  # print(line)
  output.append(line)
    
print(output)

[{'prompt': 'What Is Depression?', 'completion': 'Depression is a common and serious medical illness that negatively affects how you feel, the way you think and how you act. Fortunately,it is also treatable. Depression causes feelings of sadness and/or a loss of interest in activities you once enjoyed. It can lead to a variety of emotional and physical problems and can decrease your ability to function at work and at home.'}, {'prompt': 'I feel i have let my parents down', 'completion': 'No matter what,your parents will always be proud of you and will love you. You will feel much better if you share your feelings with them.'}, {'prompt': 'What are the types of depression?', 'completion': 'There are many different forms of depression but most common types are. Clinical depression,persistent depressive disorder,bipolar disorder,postnatal disorder.'}, {'prompt': 'I am good for nothing.', 'completion': "Don't ever think you cannot achieve what you want. You are capable of reaching your goa

In [34]:
from google.colab import files

In [ ]:
with open('depression-d.jsonl', 'w') as outfile:
        for i in output:
            json.dump(i, outfile)
            outfile.write('\n')


files.download('depression-d.jsonl')


In [36]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
!openai tools fine_tunes.prepare_data -f '/content/gdrive/MyDrive/A1b-ChatGPT/depression-d.jsonl'


Analyzing...

- Your file contains 51 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- Your data does not contain a common ending at the end of your completions. Having a common ending string appended to the end of the completion makes it clearer to the fine-tuned model where the completion should end. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples.
- The completion should start with a whitespace

In [45]:
import os

In [39]:
os.environ['OPENAI_API_KEY'] = 'insert ur api key'

In [40]:
# !set OPENAI_API_KEY=<YOUR-API-KEY>

In [41]:
!openai api fine_tunes.create -t "/content/gdrive/MyDrive/A1b-ChatGPT/depression-d_prepared.jsonl" -m davinci

Upload progress: 100% 14.8k/14.8k [00:00<00:00, 20.8Mit/s]
Uploaded file from /content/gdrive/MyDrive/A1b-ChatGPT/depression-d_prepared.jsonl: file-I6CNHjWUQfBWYIVnIojw6YHI
Created fine-tune: ft-xj5zlsU90PNwVHYanYRE63Jq
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-02-02 05:46:45] Created fine-tune: ft-xj5zlsU90PNwVHYanYRE63Jq
[2023-02-02 05:47:54] Fine-tune costs $0.37
[2023-02-02 05:47:55] Fine-tune enqueued. Queue number: 0
[2023-02-02 05:47:57] Fine-tune started
[2023-02-02 05:50:17] Completed epoch 1/4
[2023-02-02 05:50:32] Completed epoch 2/4
[2023-02-02 05:50:48] Completed epoch 3/4
[2023-02-02 05:51:04] Completed epoch 4/4
[2023-02-02 05:51:41] Uploaded model: davinci:ft-personal-2023-02-02-05-51-41
[2023-02-02 05:51:42] Uploaded result file: file-JWRjxcYYudj4HcmRqnCN5j4P
[2023-02-02 05:51:43] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.c

In [42]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

query = 'hi i feel low'



def openai_create(prompt):

    response = openai.Completion.create(
    engine="davinci:ft-personal-2023-02-02-05-51-41",   # fine tuned davinci model
    # engine="davinci",   # base engine without fine tuning
    prompt=prompt,
    temperature=1.0,
    max_tokens=162,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0.6,
    stop=["\n"]
    )

    return response.choices[0].text


In [43]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import gradio as gr

#iface = gr.Interface.load('davinci:ft-personal-2023-02-01-19-15-41','models'
#,examples=[["This covid is very real"],['tips:Label_0=Negative, Label_1=Neutral , Label_2=Positive']]
#)
#iface.launch()


demo = gr.Interface(fn=openai_create, inputs="text", outputs="text")
    
if __name__ == "__main__":
    demo.launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>